In [ ]:
#####################################
# This creates figure
# TraxAceBar.png 
# in paper
#####################################
import sys
# import modules in other directories
sys.path.append('../Utils/')

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature



import trax_util as trx
import ibtracs as IBT
import parse_tempest_file as tmpst

import importlib
import PlotUtil as Pu

importlib.reload( tmpst )
importlib.reload( Pu )
importlib.reload( trx )
importlib.reload( IBT )


In [ ]:


file = '/glade/work/stepheba/post/tempestextremes/xic_x02_ne240_pg3/trajectories.txt.julio_xic_x02'
tc2004 = tmpst.parse_storm_file(file)
# Example usage
#storm_data = parse_storm_file('path_to_your_file.txt')
print( len(tc2004))

In [ ]:
#plt.plot(tc2004[1]['wind'])
# Collect all wind data across storms
all_winds = []

for storm in tc2004:
    all_winds.extend(storm['wind'])  # Append all wind values from the current storm

# List to store the highest wind value from each storm
highest_winds = []

for storm in tc2004:
    if storm['wind']:  # Check if the wind list is not empty
        highest_winds.append(max(storm['wind']))


plt.plot(highest_winds)

In [ ]:
#f=trx.rcp85fname(ens=1,sst='sst3')
#f=trx.pdfname(ens=1)
power_wind=(1./6.)**(0.11)

pd1=trx.readtrx( trx.pdfname(ens=1) , power_wind=power_wind )
pd2=trx.readtrx( trx.pdfname(ens=2) , power_wind=power_wind )
pd3=trx.readtrx( trx.pdfname(ens=3) , power_wind=power_wind )
#ibx=IBT.readtrx(Fill_NI_before_1990=False)
ibx=IBT.readtrx()



In [ ]:
#ibx

yook = np.where( ( ibx.year[:,0] == 2010 ) )
len(yook[0])

In [ ]:
#projPC = ccrs.Robinson(central_longitude=180.)
projPC = ccrs.PlateCarree(central_longitude=180.)
data_crs = ccrs.PlateCarree()

basi = IBT.basin_outlines(Mediterranean=False,xNEPacific=True)
ipan=-1
"""
Add axes method 
ax = fig.add_axes([xmin , ymin , dx , dy , ])
"""

Axes = np.asarray( [ 
    [ .05, .05, .8, .8 ] ,
    [ .55, .6, .4, .25] ,
    [ .05, .3, .4, .25 ] ,
    [ .55, .3, .4, .25 ] ,
    [ .3, 0., .4, .25] 
    ])

print(np.shape(Axes))

print(Axes[0,:])

scale_fac = 1e-4 * 1.944**2 
denslv=21 #np.linspace(0,15,num=16)
#denslv=np.linspace(500,10_500,num=21)
denslv=np.linspace(0.2,4.0,num=20)

#scale_fac = 1.0 #1e-4 * 1.944**2 
#denslv= np.linspace(0.002,0.052,num=21) # - 0.05


fig = plt.figure(figsize=(20, 15))
levs=[29,26,20,10]
    
ax1 = fig.add_axes( Axes[0,:] , projection=projPC)
ax1.set_global()
ax1.coastlines(resolution='110m',color='black')
ax1.set_title( "Ens. member #1" , fontsize=16)

###
# Plot IBTrACS basins
###
bbox=dict(facecolor='white', edgecolor='black')
for b in np.arange( start=1, stop=len(basi) ):
    xyb=np.array( basi[b].xy )
    loos='-'
    if ((basi[b].number == 1) or (basi[b].number == 7)) :
        loos='--'
    ax1.plot( xyb[:,0],xyb[:,1] ,color='red',transform=data_crs,linestyle=loos)
    if (basi[b].labloc[0]>-9999):
        ax1.text( basi[b].labloc[0],basi[b].labloc[1],basi[b].name , transform=data_crs, color='red' , bbox=bbox ) 

ist=0
for ii in np.arange(len(yook[0])):
    ist=yook[0][ii]
    ax1.scatter( ibx.lon[ist,:],ibx.lat[ist,:],transform=data_crs, s=.2 )


In [ ]:
x=np.linspace(0,100,num=101)
iw=np.count_nonzero( x<=33) - 1
iw=np.max((0,iw))

In [ ]:
x=np.linspace(0,360,num=2*3600+1 )
y=np.linspace(-90,90,num=2*1800+1 )

x=np.linspace(0,360,num=360//2+1 )
y=np.linspace(-90,90,num=180//2+1 )

lons = 0.5*( x[1:]+x[0:-1] )
lats = 0.5*( y[1:]+y[0:-1] )

adens1 = trx.adensity(trx=pd1,x=x,y=y,alpha=2) #,category='Cat1+')
dens1 = trx.density(trx=pd1,x=x,y=y,category='Cat1+')
print('done 1')

adens2 = trx.adensity(trx=pd2,x=x,y=y,alpha=2) #,category='Cat1+')
dens2 = trx.density(trx=pd2,x=x,y=y,category='Cat1+')
print('done 2')
adens3 = trx.adensity(trx=pd3,x=x,y=y,alpha=2) #,category='Cat1+')
dens3 = trx.density(trx=pd3,x=x,y=y,category='Cat1+')
print('done 3')

Idens = trx.density(trx=ibx,x=x,y=y,category='Cat1+')
Iadens = trx.adensity(trx=ibx,x=x,y=y,alpha=2) #,category='Cat1+')

print(np.shape(dens1))
print(np.shape(x),np.shape(lons))
print(np.shape(y),np.shape(lats))

In [ ]:
print(pd1.hour[0,0:5])
print(ibx.hour[0,0:5])


print( np.asarray((500,10_500)) * 1e-4 * 1.944**2 )

In [ ]:
# Power wind law was already applied ... so set to 1 here
power_wind=1.0 # (1./6.)**(0.11)

bace1=trx.basinACE(wind=pd1.wind,basin=pd1.basin,power_wind=power_wind)
bace1=bace1 / 33.
bace2=trx.basinACE(wind=pd2.wind,basin=pd2.basin,power_wind=power_wind)
bace2=bace2 / 33.
bace3=trx.basinACE(wind=pd3.wind,basin=pd3.basin,power_wind=power_wind)
bace3=bace3 / 33.
Ibace=trx.basinACE(wind=ibx.wind,basin=ibx.basin)
Ibace=Ibace / 43.


In [ ]:
#projPC = ccrs.Robinson(central_longitude=180.)
projPC = ccrs.PlateCarree(central_longitude=180.)
data_crs = ccrs.PlateCarree()

basi = IBT.basin_outlines(Mediterranean=False,xNEPacific=True)
ipan=-1
"""
Add axes method 
ax = fig.add_axes([xmin , ymin , dx , dy , ])
"""

Axes = np.asarray( [ 
    [ .05, .6, .4, .25 ] ,
    [ .55, .6, .4, .25] ,
    [ .05, .3, .4, .25 ] ,
    [ .55, .3, .4, .25 ] ,
    [ .3, 0., .4, .25] 
    ])

print(np.shape(Axes))

print(Axes[0,:])

scale_fac = 1e-4 * 1.944**2 
denslv=21 #np.linspace(0,15,num=16)
#denslv=np.linspace(500,10_500,num=21)
denslv=np.linspace(0.2,4.0,num=20)

#scale_fac = 1.0 #1e-4 * 1.944**2 
#denslv= np.linspace(0.002,0.052,num=21) # - 0.05


fig = plt.figure(figsize=(20, 15))
levs=[29,26,20,10]
    
ax1 = fig.add_axes( Axes[0,:] , projection=projPC)
ax1.set_global()
ax1.coastlines(resolution='110m',color='black')
po=ax1.contourf(lons,lats,scale_fac*adens1/34.,transform=data_crs,levels=denslv,cmap='gist_ncar')
ax1.set_title( "Ens. member #1" , fontsize=16)

###
# Plot IBTrACS basins
###
bbox=dict(facecolor='white', edgecolor='black')
for b in np.arange( start=1, stop=len(basi) ):
    xyb=np.array( basi[b].xy )
    loos='-'
    if ((basi[b].number == 1) or (basi[b].number == 7)) :
        loos='--'
    ax1.plot( xyb[:,0],xyb[:,1] ,color='red',transform=data_crs,linestyle=loos)
    if (basi[b].labloc[0]>-9999):
        ax1.text( basi[b].labloc[0],basi[b].labloc[1],basi[b].name , transform=data_crs, color='red' , bbox=bbox ) 

ipan=ipan+1
ax1.text(-0.1, 1.1, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')
 

ax1 = fig.add_axes( Axes[1,:] , projection=projPC)
ax1.set_global()
ax1.coastlines(resolution='110m',color='black')
po=ax1.contourf(lons,lats,scale_fac*adens2/34.,transform=data_crs,levels=denslv,cmap='gist_ncar')
ax1.set_title( "Ens. member #2" , fontsize=16)
ipan=ipan+1
ax1.text(-0.1, 1.1, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

ax1 = fig.add_axes( Axes[2,:] , projection=projPC)
ax1.set_global()
ax1.coastlines(resolution='110m',color='black')
po=ax1.contourf(lons,lats,scale_fac*adens3/34.,transform=data_crs,levels=denslv,cmap='gist_ncar')
ax1.set_title( "Ens. member #3" , fontsize=16)
ipan=ipan+1
ax1.text(-0.1, 1.1, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')


ax1 = fig.add_axes( Axes[3,:] , projection=projPC)
ax1.set_global()
ax1.coastlines(resolution='110m',color='black')
po=ax1.contourf(lons,lats,scale_fac*Iadens/44.,transform=data_crs,levels=denslv,cmap='gist_ncar')
ax1.set_title( "IBTrACS" , fontsize=16)
ipan=ipan+1
ax1.text(-0.1, 1.1, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

"""
###
# Plot IBTrACS basins
###
bbox=dict(facecolor='white', edgecolor='black')
for b in np.arange( start=1, stop=len(basi) ):
    xyb=np.array( basi[b].xy )
    ax1.plot( xyb[:,0],xyb[:,1] ,color='red',transform=data_crs)
    if (basi[b].labloc[0]>-9999):
        ax1.text( basi[b].labloc[0],basi[b].labloc[1],basi[b].name , transform=data_crs, color='red' , bbox=bbox ) 
"""


#####################################
# ACE bars
#####################################

BarLabel = "ACE year$^{-1}$ (2$^{\circ}$)$^{-2}$"
cax = plt.axes([0.98, 0.35, 0.02, 0.4])
cbar = plt.colorbar(po,cax=cax,orientation='vertical')
cbar.set_label( label=BarLabel,size=16)
cbar.ax.tick_params(labelsize=12)



ax1 = fig.add_axes( Axes[4,:]  )

# Get some pastel shades for the bar colors
colors = plt.cm.BuPu(np.linspace(0.2, 0.5, 3))

#boo=np.arange(8)
boo=np.arange(8)
bnames=['None','N \nAtl','S \n Atl' , 'NW \n Pac', 'NE \n Pac', 'SW \n Pac','N \n Indian','S \nIndian']
#bnames=['N \nAtl','S \n Atl' , 'NW \n Pac', 'NE \n Pac', 'SW \n Pac','N \n Indian','S \nIndian']
#plt.bar( boo,np.sum(bace,axis=1),tick_label=bnames,label=bnames)
ax1.bar( boo-.4,bace1,width=0.2,color=colors[0],label='CAM #1')
ax1.bar( boo-.2,bace2,width=0.2,color=colors[1],label='CAM #2')
ax1.bar( boo   ,bace3,width=0.2,color=colors[2] , tick_label=bnames,label='CAM #3')

ipan=ipan+1
ax1.text(-0.15, 1.1, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

#plt.bar( boo+.2,Ibace,tick_label=bnames,label=bnames,width=0.2,color='orange')
ax1.bar( boo+.2,Ibace,width=0.2,color='red',label='IBTrACS')
plt.legend()
plt.xlim(.4,8)
plt.ylabel("1980-2012 Accumulated Cyclone Energy",size=16)

Pu.save_link(png_filename = "TraxAceBar.png")

In [ ]:
#projPC = ccrs.Robinson(central_longitude=180.)
projPC = ccrs.PlateCarree(central_longitude=180.)
data_crs = ccrs.PlateCarree()

basi = IBT.basin_outlines(Mediterranean=False,xNEPacific=True)
ipan=-1
"""
Add axes method 
ax = fig.add_axes([xmin , ymin , dx , dy , ])
"""

Axes = np.asarray( [ 
    [ .05, .6, .4, .25 ] ,
    [ .55, .6, .4, .25] ,
    [ .05, .3, .4, .25 ] ,
    [ .55, .3, .4, .25 ] ,
    [ .3, 0., .4, .25] 
    ])

print(np.shape(Axes))

print(Axes[0,:])


scale_fac = 3.0 #1e-4 * 1.944**2 
denslv= 100.*np.linspace(0.002,0.052,num=21) # - 0.05


fig = plt.figure(figsize=(20, 15))
levs=[29,26,20,10]
    
ax1 = fig.add_axes( Axes[0,:] , projection=projPC)
ax1.set_global()
ax1.coastlines(resolution='110m',color='black')
po=ax1.contourf(lons,lats,scale_fac*dens1,transform=data_crs,levels=denslv,cmap='gist_ncar')
ax1.set_title( "Ens. member #1" , fontsize=16)

###
# Plot IBTrACS basins
###
bbox=dict(facecolor='white', edgecolor='black')
for b in np.arange( start=1, stop=len(basi) ):
    xyb=np.array( basi[b].xy )
    loos='-'
    if ((basi[b].number == 1) or (basi[b].number == 7)) :
        loos='--'
    ax1.plot( xyb[:,0],xyb[:,1] ,color='red',transform=data_crs,linestyle=loos)
    if (basi[b].labloc[0]>-9999):
        ax1.text( basi[b].labloc[0],basi[b].labloc[1],basi[b].name , transform=data_crs, color='red' , bbox=bbox ) 

ipan=ipan+1
ax1.text(-0.1, 1.1, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')
 

ax1 = fig.add_axes( Axes[1,:] , projection=projPC)
ax1.set_global()
ax1.coastlines(resolution='110m',color='black')
po=ax1.contourf(lons,lats,scale_fac*dens2,transform=data_crs,levels=denslv,cmap='gist_ncar')
ax1.set_title( "Ens. member #2" , fontsize=16)
ipan=ipan+1
ax1.text(-0.1, 1.1, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

ax1 = fig.add_axes( Axes[2,:] , projection=projPC)
ax1.set_global()
ax1.coastlines(resolution='110m',color='black')
po=ax1.contourf(lons,lats,scale_fac*dens3,transform=data_crs,levels=denslv,cmap='gist_ncar')
ax1.set_title( "Ens. member #3" , fontsize=16)
ipan=ipan+1
ax1.text(-0.1, 1.1, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')


ax1 = fig.add_axes( Axes[3,:] , projection=projPC)
ax1.set_global()
ax1.coastlines(resolution='110m',color='black')
po=ax1.contourf(lons,lats,scale_fac*Idens,transform=data_crs,levels=denslv,cmap='gist_ncar')
ax1.set_title( "IBTrACS" , fontsize=16)
ipan=ipan+1
ax1.text(-0.1, 1.1, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

"""
###
# Plot IBTrACS basins
###
bbox=dict(facecolor='white', edgecolor='black')
for b in np.arange( start=1, stop=len(basi) ):
    xyb=np.array( basi[b].xy )
    ax1.plot( xyb[:,0],xyb[:,1] ,color='red',transform=data_crs)
    if (basi[b].labloc[0]>-9999):
        ax1.text( basi[b].labloc[0],basi[b].labloc[1],basi[b].name , transform=data_crs, color='red' , bbox=bbox ) 
"""


#####################################
# ACE bars
#####################################

BarLabel = "Hrs(Cat1+) year$^{-1}$ (2$^{\circ}$)$^{-2}$"
cax = plt.axes([0.98, 0.35, 0.02, 0.4])
cbar = plt.colorbar(po,cax=cax,orientation='vertical')
cbar.set_label( label=BarLabel,size=16)
cbar.ax.tick_params(labelsize=12)



ax1 = fig.add_axes( Axes[4,:]  )

# Get some pastel shades for the bar colors
colors = plt.cm.BuPu(np.linspace(0.2, 0.5, 3))

#boo=np.arange(8)
boo=np.arange(8)
bnames=['None','N \nAtl','S \n Atl' , 'NW \n Pac', 'NE \n Pac', 'SW \n Pac','N \n Indian','S \nIndian']
#bnames=['N \nAtl','S \n Atl' , 'NW \n Pac', 'NE \n Pac', 'SW \n Pac','N \n Indian','S \nIndian']
#plt.bar( boo,np.sum(bace,axis=1),tick_label=bnames,label=bnames)
ax1.bar( boo-.4,bace1,width=0.2,color=colors[0],label='CAM #1')
ax1.bar( boo-.2,bace2,width=0.2,color=colors[1],label='CAM #2')
ax1.bar( boo   ,bace3,width=0.2,color=colors[2] , tick_label=bnames,label='CAM #3')

ipan=ipan+1
ax1.text(-0.15, 1.1, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

#plt.bar( boo+.2,Ibace,tick_label=bnames,label=bnames,width=0.2,color='orange')
ax1.bar( boo+.2,Ibace,width=0.2,color='red',label='IBTrACS')
plt.legend()
plt.xlim(.4,8)
plt.ylabel("1980-2012 Accumulated Cyclone Energy",size=16)

Pu.save_link(png_filename = "TraxCat1HrsBar.png")

In [ ]:

lonW = -110
lonE = -10
latS = 5
latN = 45
cLat = (latN + latS) / 2
cLon = (lonW + lonE) / 2
res = '110m'

#projPC = ccrs.Robinson(central_longitude=180.)
data_crs = ccrs.PlateCarree()

projPC = ccrs.Stereographic(central_longitude=cLon, central_latitude=cLat)
"""
Add axes method 
ax = fig.add_axes([xmin , ymin , dx , dy , ])
"""

Axes = np.asarray( [ 
    [ .05, .6, .4, .25 ] ,
    [ .55, .6, .4, .25] ,
    [ .05, .3, .4, .25 ] ,
    [ .55, .3, .4, .25 ] ,
    [ .3, 0., .4, .25] 
    ])

print(np.shape(Axes))

print(Axes[0,:])

scale_fac = 1e-4 * 1.944**2 
denslv=21 #np.linspace(0,15,num=16)
#denslv=np.linspace(500,10_500,num=21)
denslv=np.linspace(0.1,2.0,num=20) - 0.05

fig = plt.figure(figsize=(20, 15))
levs=[29,26,20,10]
    
ax1 = fig.add_axes( Axes[0,:] , projection=projPC)
ax1.set_global()
ax1.coastlines(resolution='110m',color='gray',linewidth=2)
po=ax1.contourf(lons,lats,scale_fac*adens1/34.,transform=data_crs,levels=denslv,cmap='gist_ncar')
ax1.set_title( "Ens. member #1" , fontsize=16)
ax1.set_extent([lonW, lonE, latS, latN], crs=data_crs)

ax1 = fig.add_axes( Axes[1,:] , projection=projPC)
ax1.set_global()
ax1.coastlines(resolution='110m',color='gray',linewidth=2)
#ax1.coastlines(resolution='110m',color='black')
po=ax1.contourf(lons,lats,scale_fac*adens2/34.,transform=data_crs,levels=denslv,cmap='gist_ncar')
ax1.set_title( "Ens. member #2" , fontsize=16)
ax1.set_extent([lonW, lonE, latS, latN], crs=data_crs)

ax1 = fig.add_axes( Axes[2,:] , projection=projPC)
ax1.set_global()
ax1.coastlines(resolution='110m',color='gray',linewidth=2)
#ax1.coastlines(resolution='110m',color='black')
po=ax1.contourf(lons,lats,scale_fac*adens3/34.,transform=data_crs,levels=denslv,cmap='gist_ncar')
ax1.set_title( "Ens. member #3" , fontsize=16)
ax1.set_extent([lonW, lonE, latS, latN], crs=data_crs)

ax1 = fig.add_axes( Axes[3,:] , projection=projPC)
ax1.set_global()
ax1.coastlines(resolution='110m',color='gray',linewidth=2)
#ax1.coastlines(resolution='110m',color='black')
po=ax1.contourf(lons,lats,scale_fac*Iadens/44.,transform=data_crs,levels=denslv,cmap='gist_ncar')
ax1.set_title( "IBTrACS" , fontsize=16)
ax1.set_extent([lonW, lonE, latS, latN], crs=data_crs)

BarLabel = "ACE year$^{-1}$ (2$^{\circ}$)$^{-2}$"
cax = plt.axes([0.98, 0.35, 0.02, 0.4])
cbar = plt.colorbar(po,cax=cax,orientation='vertical')
cbar.set_label( label=BarLabel,size=16)
cbar.ax.tick_params(labelsize=12)


ax1 = fig.add_axes( Axes[4,:]  )

# Get some pastel shades for the colors
colors = plt.cm.BuPu(np.linspace(0.2, 0.5, 3))

#boo=np.arange(8)
boo=np.arange(8)
bnames=['None','N \nAtl','S \n Atl' , 'NW \n Pac', 'NE \n Pac', 'SW \n Pac','N \n Indian','S \nIndian']
#bnames=['N \nAtl','S \n Atl' , 'NW \n Pac', 'NE \n Pac', 'SW \n Pac','N \n Indian','S \nIndian']
#plt.bar( boo,np.sum(bace,axis=1),tick_label=bnames,label=bnames)
ax1.bar( boo-.4,bace1,width=0.2,color=colors[0],label='CAM #1')
ax1.bar( boo-.2,bace2,width=0.2,color=colors[1],label='CAM #2')
ax1.bar( boo   ,bace3,width=0.2,color=colors[2] , tick_label=bnames,label='CAM #3')


#plt.bar( boo+.2,Ibace,tick_label=bnames,label=bnames,width=0.2,color='orange')
ax1.bar( boo+.2,Ibace,width=0.2,color='red',label='IBTrACS')
plt.legend()
plt.xlim(.4,8)
plt.ylabel("1980-2012 Accumulated Cyclone Energy",size=16)


In [ ]:

lonW = -110
lonE = -10
latS = 5
latN = 45
cLat = (latN + latS) / 2
cLon = (lonW + lonE) / 2
res = '110m'

#projPC = ccrs.Robinson(central_longitude=180.)
dataProj = ccrs.PlateCarree()

MapProj = ccrs.Stereographic(central_longitude=cLon, central_latitude=cLat)

adensE = ( adens1+adens2+adens3 )/3.

n=0
nxplo,nyplo=2,1


scale_fac = 1e-4 * 1.944**2 
denslv=21 #np.linspace(0,15,num=16)
#denslv=np.linspace(500,10_500,num=21)
denslv=np.linspace(0.1,2.0,num=20) - 0.05

fig = plt.figure(figsize=(20, 15))
levs=[29,26,20,10]
    
    
n=n+1
Axes = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo)
ax1 = fig.add_axes( Axes, projection=MapProj)
ax1.set_global()
ax1.coastlines(resolution='110m',color='gray',linewidth=2)
po=ax1.contourf(lons,lats,scale_fac*adensE/34.,transform=dataProj,levels=denslv,cmap='gist_ncar')
ax1.set_title( "PD Ensemble Mean" , fontsize=16)
ax1.set_extent([lonW, lonE, latS, latN], crs=dataProj)
ax1.text(-0.15, 1.1, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')


n=n+1
Axes = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo)
ax1 = fig.add_axes( Axes , projection=MapProj)
ax1.set_global()
ax1.coastlines(resolution='110m',color='gray',linewidth=2)
#ax1.coastlines(resolution='110m',color='black')
po=ax1.contourf(lons,lats,scale_fac*Iadens/44.,transform=dataProj,levels=denslv,cmap='gist_ncar')
ax1.set_title( "IBTrACS" , fontsize=16)
ax1.set_extent([lonW, lonE, latS, latN], crs=dataProj)
ax1.text(-0.15, 1.1, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

BarLabel = "ACE year$^{-1}$ (2$^{\circ}$)$^{-2}$"
cax = plt.axes([0.98, 0.3, 0.03, 0.3])
cbar = plt.colorbar(po,cax=cax,orientation='vertical')
cbar.set_label( label=BarLabel,size=16)
cbar.ax.tick_params(labelsize=12)


Pu.save_link(png_filename = "NAtlZoom.png")

In [ ]:

lonW = -110
lonE = -10
latS = 5
latN = 45
cLat = (latN + latS) / 2
cLon = (lonW + lonE) / 2
res = '110m'

#projPC = ccrs.Robinson(central_longitude=180.)
dataProj = ccrs.PlateCarree()

MapProj = ccrs.Stereographic(central_longitude=cLon, central_latitude=cLat)

densE = ( dens1+dens2+dens3 )/3.

n=0
nxplo,nyplo=2,1


scale_fac = 3.0 #1e-4 * 1.944**2 
denslv= 100*np.linspace(0.002,0.052,num=21) # - 0.05

fig = plt.figure(figsize=(20, 15))
levs=[29,26,20,10]
    
    
n=n+1
Axes = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo)
ax1 = fig.add_axes( Axes, projection=MapProj)
ax1.set_global()
ax1.coastlines(resolution='110m',color='gray',linewidth=2)
po=ax1.contourf(lons,lats,scale_fac*densE ,transform=dataProj,levels=denslv,cmap='gist_ncar')
ax1.set_title( "PD Ensemble Mean" , fontsize=16)
ax1.set_extent([lonW, lonE, latS, latN], crs=dataProj)
ax1.text(-0.15, 1.1, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')


n=n+1
Axes = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo)
ax1 = fig.add_axes( Axes , projection=MapProj)
ax1.set_global()
ax1.coastlines(resolution='110m',color='gray',linewidth=2)
#ax1.coastlines(resolution='110m',color='black')
po=ax1.contourf(lons,lats,scale_fac*Idens,transform=dataProj,levels=denslv,cmap='gist_ncar')
ax1.set_title( "IBTrACS" , fontsize=16)
ax1.set_extent([lonW, lonE, latS, latN], crs=dataProj)
ax1.text(-0.15, 1.1, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

BarLabel = "Hrs(Cat1+) year$^{-1}$ (2$^{\circ}$)$^{-2}$"
cax = plt.axes([0.98, 0.3, 0.03, 0.3])
cbar = plt.colorbar(po,cax=cax,orientation='vertical')
cbar.set_label( label=BarLabel,size=16)
cbar.ax.tick_params(labelsize=12)


Pu.save_link(png_filename = "NAtlCat1HrsZoom.png")